<h2>Policy evaluation with Frozen Lake</h2>

We can explore the same random step process through gridworld (or in the case of OpenGym 'Frozen Lake'). Where the goal is to get from S (start) to G (goal) stepping only on safe F (frozen) squares and avoiding H (holes)

In [12]:
import gym
import numpy as np

First, let's explore the properties of this environment...

In [4]:
env=gym.make('FrozenLake-v0')
obs = env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


Final state is named '15'. Rew is +1 when you get here and 0 everywhere else. The episode wil l terminate if you reach the end, or fall in a hole

In [5]:
P = env.env.P #transition probs P[s][a]
P[14] #Prob, next state, reward, done

{0: [(0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 13, 0.0, False),
  (0.3333333333333333, 14, 0.0, False)],
 1: [(0.3333333333333333, 13, 0.0, False),
  (0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 15, 1.0, True)],
 2: [(0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 15, 1.0, True),
  (0.3333333333333333, 10, 0.0, False)],
 3: [(0.3333333333333333, 15, 1.0, True),
  (0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 13, 0.0, False)]}

<b>Example:stepping through env with a random policy</b>

In [7]:
for i_ep in range(10):
    obs = env.reset() #initial obs
    for t in range(10):
        env.render()
        act = env.action_space.sample()
        obs, rew, done, info = env.step(act) #take a random action
        print('obs',obs, 'rew: ',rew, done, info)


SFFF
FHFH
FFFH
HFFG
obs 0 rew:  0.0 False {'prob': 0.3333333333333333}
  (Left)
SFFF
FHFH
FFFH
HFFG
obs 0 rew:  0.0 False {'prob': 0.3333333333333333}
  (Up)
SFFF
FHFH
FFFH
HFFG
obs 4 rew:  0.0 False {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
obs 4 rew:  0.0 False {'prob': 0.3333333333333333}
  (Left)
SFFF
FHFH
FFFH
HFFG
obs 5 rew:  0.0 True {'prob': 0.3333333333333333}
  (Up)
SFFF
FHFH
FFFH
HFFG
obs 5 rew:  0 True {'prob': 1.0}
  (Up)
SFFF
FHFH
FFFH
HFFG
obs 5 rew:  0 True {'prob': 1.0}
  (Up)
SFFF
FHFH
FFFH
HFFG
obs 5 rew:  0 True {'prob': 1.0}
  (Up)
SFFF
FHFH
FFFH
HFFG
obs 5 rew:  0 True {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG
obs 5 rew:  0 True {'prob': 1.0}

SFFF
FHFH
FFFH
HFFG
obs 1 rew:  0.0 False {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
obs 1 rew:  0.0 False {'prob': 0.3333333333333333}
  (Right)
SFFF
FHFH
FFFH
HFFG
obs 5 rew:  0.0 True {'prob': 0.3333333333333333}
  (Left)
SFFF
FHFH
FFFH
HFFG
obs 5 rew:  0 True {'prob': 1.0}
  (Up)
SFFF
FHF

<b>Policy evaluation</b>

Dynamic programming (DP) requires that a full backup is made. This requires full knowledge of our environment.

In [9]:
def policy_eval(policy, env, num_steps, lambd=0.9):
    V = np.zeros(env.env.nS) #grid_h * grid_w values
    delta = np.zeros(env.env.nS) #grid_h * grid_w values
    print('Sum of deltas:')
    for step in range(num_steps):
        for s in range(env.env.nS): #for each state do full backup
            v=0
            for a, action_p in enumerate(policy[s]): #from that state, for all possible actions
                for prob, next_state, rew, done in env.env.P[s][a]: #for all next states
                    v+= action_p * prob * (rew + lambd*V[next_state]) # pol(s|a)* (rew* disc(sum(prob of ss * val ss))) 
            
            
            delta[s] =  np.abs(v-V[s])
            V[s] = v
        print("Step %d:"%(step+1), sum(delta))
        if sum(delta)<0.0001:
                break
    return V

In [13]:
rand_pol = np.ones([env.env.nS, env.env.nA])/env.env.nA

We can now examine the value returned after 1, 5 and 30 steps

In [14]:
val = policy_eval(rand_pol, env, 1)
print(val.reshape((4,4)))

Sum of deltas:
Step 1: 0.25
[[ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.25  0.  ]]


In [15]:
val = policy_eval(rand_pol, env, 5)
print(val.reshape((4,4)))

Sum of deltas:
Step 1: 0.25
Step 2: 0.1940625
Step 3: 0.134227441406
Step 4: 0.0773750028076
Step 5: 0.0437473326036
[[ 0.          0.00064072  0.0052824   0.00147687]
 [ 0.00128145  0.          0.0226776   0.        ]
 [ 0.01077304  0.04912829  0.10135362  0.        ]
 [ 0.          0.12146647  0.38533182  0.        ]]


In [16]:
val = policy_eval(rand_pol, env, 15)
print(val.reshape((4,4)))

Sum of deltas:
Step 1: 0.25
Step 2: 0.1940625
Step 3: 0.134227441406
Step 4: 0.0773750028076
Step 5: 0.0437473326036
Step 6: 0.0249144380738
Step 7: 0.0144215946486
Step 8: 0.00851521504157
Step 9: 0.00513165495994
Step 10: 0.00315247107891
Step 11: 0.00196959923813
Step 12: 0.00124812495468
Step 13: 0.0008000496388
Step 14: 0.000517479310903
Step 15: 0.00033704370329
[[ 0.00427519  0.00411627  0.00999268  0.00406645]
 [ 0.00662951  0.          0.02631215  0.        ]
 [ 0.01863237  0.05758692  0.10695876  0.        ]
 [ 0.          0.13036937  0.39148029  0.        ]]


In [17]:
val = policy_eval(rand_pol, env, 30)
print(val.reshape((4,4)))

Sum of deltas:
Step 1: 0.25
Step 2: 0.1940625
Step 3: 0.134227441406
Step 4: 0.0773750028076
Step 5: 0.0437473326036
Step 6: 0.0249144380738
Step 7: 0.0144215946486
Step 8: 0.00851521504157
Step 9: 0.00513165495994
Step 10: 0.00315247107891
Step 11: 0.00196959923813
Step 12: 0.00124812495468
Step 13: 0.0008000496388
Step 14: 0.000517479310903
Step 15: 0.00033704370329
Step 16: 0.000220681732508
Step 17: 0.000145063883947
Step 18: 9.56364702317e-05
[[ 0.00441661  0.00419078  0.01004509  0.00410308]
 [ 0.00669509  0.          0.02632768  0.        ]
 [ 0.01866426  0.05760208  0.10696868  0.        ]
 [ 0.          0.13038004  0.39148794  0.        ]]


Running for 30 time steps, we can see that the values converged after 18 time steps (as the sum of all changes was less than the pre-defined boundary)